In [ ]:
import sys
sys.path.append('..')
from src.networks import BaseNetwork
import torch
import torch_geometric as pyg

In [22]:
import torch


class TransformerDecoder(torch.nn.Module):
    def __init__(self, latent_dim, context_dim, hidden_dim, output_dim, n_heads, n_layers):
        super().__init__()
        self.embedding = torch.nn.Sequential(
            torch.nn.Linear(latent_dim + context_dim, hidden_dim),
            torch.nn.Dropout(0.3),
            torch.nn.LeakyReLU(0.1)
        )
#        self.decoder = torch.nn.TransformerDecoder(
#            torch.nn.TransformerDecoderLayer(
#                hidden_dim, n_heads, dim_feedforward=hidden_dim * 4
#            ), n_layers
#        )
        self.decoder = torch.nn.TransformerDecoderLayer(
            d_model=hidden_dim, nhead=n_heads, dim_feedforward=hidden_dim * 4
        )
        self.output = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, z, context, target_seq):
        n_batch, n_seq, _ = z.shape
        c = torch.concat([z, context.unsqueeze(1).repeat((1, n_seq, 1))], dim=-1)
        print(z.shape, context.shape, target_seq.shape)
        print(c.shape)
        mem = self.embedding(c)
        print(mem.shape, target_seq.shape)
        out = self.decoder(target_seq.transpose(0,1), mem.transpose(0,1))
        print(out.shape)
        return self.output(out.transpose(0,1))

dec = TransformerDecoder(
    latent_dim=12, context_dim=44, hidden_dim=32, output_dim=100, n_heads=8, n_layers=4
)
z = torch.rand((4,10,12))
context = torch.rand((4, 44))
target = torch.tensor([
    [0, 4, 5, 2, 8, 0, 0],
    [0, 2, 4, 7, 3, 0, 0],
    [0, 1, 8, 6, 3, 4, 0],
    [0, 1, 5, 0, 0, 0, 0]
]).long()

target_dict = torch.tensor([
    [0],
    [1],
    [2],
    [3],
    [4],
    [5],
    [6],
    [7],
    [8],
    [9],
]).repeat((1, 32)).float()

target_vec = target_dict[target]

o = dec(z, context, target_vec)
o.argmax()


torch.Size([4, 10, 12]) torch.Size([4, 44]) torch.Size([4, 7, 32])
torch.Size([4, 10, 56])
torch.Size([4, 10, 32]) torch.Size([4, 7, 32])
torch.Size([7, 4, 32])


torch.Size([4, 7, 100])

In [29]:
torch.nn.Transformer.generate_square_subsequent_mask(7)

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0.]])